In [102]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder

In [14]:
df = pd.read_csv('/content/PlayTennis.csv')
df

,Outlook,Temperature,Humidity,Wind,PlayTennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [180]:
counts = df['PlayTennis'].value_counts()
counts

PlayTennis
Yes    9
No     5
Name: count, dtype: int64

In [195]:
class naive_bayes():
  def __init__(self, df):
        self.df = df

  def calculate_probabilities(self):

      self.probabilities = {}
      for column in self.df.columns[:-1]:
          probabilities = self.df.groupby([column, 'PlayTennis']).size().unstack('PlayTennis', fill_value=0)
          #print(probabilities)
          #print(probabilities.sum(axis=0))
          probabilities = probabilities.div(probabilities.sum(axis=0), axis=1)
          self.probabilities[column] = probabilities


  def predict(self, user_outlook, user_temperature, user_humidity, user_wind):

        outlook_prob = self.probabilities['Outlook']
        temperature_prob = self.probabilities['Temperature']
        humidity_prob = self.probabilities['Humidity']
        wind_prob = self.probabilities['Wind']

        #print(outlook_prob)
        #print(temperature_prob)
        #print(humidity_prob)
        #print(wind_prob)

        prob_yes = outlook_prob.loc[user_outlook, 'Yes'] * temperature_prob.loc[user_temperature, 'Yes'] * humidity_prob.loc[user_humidity, 'Yes'] * wind_prob.loc[user_wind, 'Yes']

        prob_no = outlook_prob.loc[user_outlook, 'No'] * temperature_prob.loc[user_temperature, 'No'] * humidity_prob.loc[user_humidity, 'No'] * wind_prob.loc[user_wind, 'No']


        prob_no *= counts['No']/(counts['Yes']+counts['No'])
        prob_yes *= counts['Yes']/(counts['Yes']+counts['No'])




        return prob_yes, prob_no





PlayTennis
Yes    9
No     5
Name: count, dtype: int64

In [42]:
User_Outlook = input("Enter the outlook: ")

User_Temperature = input("Enter the temperature: ")

User_Humidity = input("Enter the humidity: ")

User_Wind = input("Enter the wind speed: ")

print("User_Outlook:", User_Outlook)
print("User_Temperature:", User_Temperature)
print("User_Humidity:", User_Humidity)
print("User_Wind:", User_Wind)

Enter the outlook: Sunny
Enter the temperature: Hot
Enter the humidity: High
Enter the wind speed: Weak
User_Outlook: Sunny
User_Temperature: Hot
User_Humidity: High
User_Wind: Weak


In [196]:
naive_prob = naive_bayes(df)
naive_prob.calculate_probabilities()

In [134]:
naive_prob.predict(df['Outlook'][2],df['Temperature'][2],df['Humidity'][2],df['Wind'][2])

PlayTennis   No       Yes
Outlook                  
Overcast    0.0  0.444444
Rain        0.4  0.333333
Sunny       0.6  0.222222
PlayTennis    No       Yes
Temperature               
Cool         0.2  0.333333
Hot          0.4  0.222222
Mild         0.4  0.444444
PlayTennis   No       Yes
Humidity                 
High        0.8  0.333333
Normal      0.2  0.666667
PlayTennis   No       Yes
Wind                     
Strong      0.6  0.333333
Weak        0.4  0.666667


(0.04938271604938271, 0.12800000000000003)

In [198]:
naive_prob.predict(User_Outlook,User_Temperature,User_Humidity,User_Wind) # returns prob yes , prob no

(0.007054673721340387, 0.02742857142857143)

In [199]:
predic=[]
for i in range(df['Outlook'].shape[0]):
  naive_prob_yes,naive_prob_no = naive_prob.predict(df['Outlook'][i],df['Temperature'][i],df['Humidity'][i],df['Wind'][i])
  #print(df['Temperature'][i])
  print(naive_prob_yes)
  print(naive_prob_no)
  print("  _  ")

  if naive_prob_yes > naive_prob_no:
    predic.append(['Yes'])
  else:
    predic.append(['No'])

0.007054673721340387
0.02742857142857143
  _  
0.0035273368606701934
0.04114285714285714
  _  
0.014109347442680773
0.0
  _  
0.021164021164021163
0.01828571428571429
  _  
0.031746031746031744
0.0022857142857142863
  _  
0.015873015873015872
0.0034285714285714297
  _  
0.021164021164021163
0.0
  _  
0.014109347442680773
0.02742857142857143
  _  
0.021164021164021163
0.003428571428571429
  _  
0.042328042328042326
0.004571428571428573
  _  
0.014109347442680773
0.010285714285714285
  _  
0.014109347442680773
0.0
  _  
0.028218694885361547
0.0
  _  
0.010582010582010581
0.027428571428571438
  _  


In [200]:
predic

[['No'],
 ['No'],
 ['Yes'],
 ['Yes'],
 ['Yes'],
 ['Yes'],
 ['Yes'],
 ['No'],
 ['Yes'],
 ['Yes'],
 ['Yes'],
 ['Yes'],
 ['Yes'],
 ['No']]

In [201]:
print(df['PlayTennis'])

0      No
1      No
2     Yes
3     Yes
4     Yes
5      No
6     Yes
7      No
8     Yes
9     Yes
10    Yes
11    Yes
12    Yes
13     No
Name: PlayTennis, dtype: object


In [202]:
# Calculate accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(df['PlayTennis'], predic)

print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 92.86%


In [203]:
Outlook = df.groupby(['Outlook', 'PlayTennis']).size()
Outlook

Outlook   PlayTennis
Overcast  Yes           4
Rain      No            2
          Yes           3
Sunny     No            3
          Yes           2
dtype: int64